# VSMC with Normalizing Flows

### To do:
- [ ] Think about how to take into consideration `x_prev`
- [ ] Implement it!

In [7]:
# Import normalizing flows module
import sys
sys.path.append("./src")
from flows import *

In [8]:
# Import other libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter

In [9]:
normal = distributions.Normal(0, 1)
flow = RealNVP(dim=2, n_layers=5, base_dist=normal)

In [6]:
# Not trained yet
flow.log_prob(torch.tensor([[0.0]]))

tensor([[-1.1349, -1.1190]], grad_fn=<AddBackward0>)

In [ ]:
def proposal_logpdf(x_t, x_prev, flow):
    flow.log_prob(x_t)
    # finish this

def vsmc_nf(y_seq, lambda_, N, T):
    latent_dim = y_seq[0].size(0)
    log_w = torch.zeros(T, N)
    x_particles = torch.zeros(T, N, latent_dim)
    ancestors = torch.zeros(T, N, dtype=torch.long)

    eps = torch.randn(N, latent_dim)
    x_t = proposal(None, eps, lambda_)
    x_particles[0] = x_t
    log_w[0] = (
        prior(x_t, None) + likelihood(y_seq[0], x_t) - proposal_logpdf(x_t, None, flow)
    ).squeeze()

    for t in range(1, T):
        w_prev = torch.softmax(log_w[t - 1], dim=0)
        # print(w_prev)
        a_t = torch.multinomial(w_prev, N, replacement=True)
        ancestors[t] = a_t

        x_prev = x_particles[t - 1, a_t]
        eps = torch.randn(N, latent_dim)
        x_t = proposal(x_prev, eps, lambda_)
        x_particles[t] = x_t

        log_w[t] = (
            prior(x_t, x_prev) + likelihood(y_seq[t], x_t) - proposal_logpdf(x_t, x_prev, flow)
        ).squeeze()

    # Sample final trajectory
    w_T = torch.softmax(log_w[-1], dim=0)
    b_T = torch.multinomial(w_T, 1).item()
    
    # Trace back trajectory
    x_estimates = torch.zeros_like(x_particles[:, 0])
    # i = b_T
    for t in reversed(range(T)):
        x_estimates[t] = x_particles[t, b_T]
        if t > 0:
            i = ancestors[t, b_T]
    
    return log_w, x_particles, ancestors, x_estimates
    # return log_w, x_particles

